## Load Packages

In [1]:
using JuMP;
using HiGHS;
using Plots;
using ParametricOptInterface;
const POI = ParametricOptInterface;

# Farmer Problem

João is a farmer from a small town who specializes in growing wheat, corn and sugar beet. He owns $500 km^2$ of land and must decide the amount of land to be allocated to each of the crops. João faces several restrictions regarding his planting. First, he must have at least $200 T$ of wheat and $240 T$ of corn to feed his cattle. Such quantities can be obtained through own plantation or by buying from the city's cooperative. The purchase prices per ton of wheat are $238 R\$/T$ and per ton of corn $210 R\$/T$. On the other hand, any excess produced in relation to the minimum can be sold at the cooperative, however with a $40\%$ discount on the purchase price ($170 R\$/T$ for wheat and $150 R\$/T$ for corn) per the cooperative's margin account. Another important restriction concerns the sale of sugar beet. By law, the sale price of a ton of beet at the cooperative is fixed at $36 R\$/T$ for the first $6000 T$ sold. After this amount, the sale price becomes $10 R\$/T$. In addition, the cooperative where João trades his products does not have sugar beet for purchase.

Let's assume that the planting cost of each crop is: $150 R\$/km^2$ for wheat, $230 R\$/km^2$ for corn, and $260 R\$/km^2$ for sugar beet. The uncertainty of the problem lies in the productivity of the land. João does not know a priori how much each $km^2$ of land will yield in tons of culture. Now assume that three scenarios of equal probabilities of occurrence were sampled: "good", "average" and "poor". In each of the states, the yield of each crop is given by:
"Bad" state: Wheat = $2 T/km^2$; Corn = $2.4 T/km^2$; Beetroot = $16 T/km^2$;
"Average" state: Wheat = $2.5 T/km^2$; Corn = $3.0 T/km^2$; Beetroot = $20 T/km^2$;
"Good" state: Wheat = $3 T/km^2$; Corn = $3.6 T/km^2$; Beetroot = $24 T/km^2$;

$$
\begin{aligned}
    &\min && 150a_1 + 230a_2 + 260a_3 + \mathbb{E}_s[Q(a,s)]  \\
    & st && a_1+a_2+a_3 \leq 500\\
        &&& 0 \leq a_1, a_2, a_3 \\
\end{aligned}
$$

$$
\begin{aligned}
   Q(a,s) = &\min && 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3   \\
    & st && p_1 + c_1 - v_1 \geq 200\\
        &&& p_2 + c_2 - v_2 \geq 240\\
        &&& l_3 \leq 36p_3\\
        &&& l_3 \leq 10(p_3-6000) + 36*6000\\
        &&& p_1 = a_1s_1\\
        &&& p_2 = a_2s_2\\
        &&& p_3 = a_3s_3\\
        &&& 0 \leq p_1, p_2, p_3\\
        &&& 0 \leq c_1, c_2, v_1, v_2\\
        &&& 0 \leq l_3
\end{aligned}
$$

## 1) Compute and plot the 2 stage function 

In [23]:
function create_farmer_main()
    
    main = Model(HiGHS.Optimizer)
    MOI.set(main, JuMP.MOI.Silent(), true)
    @variable(main, 0<=a[1:3])
    @variable(main, -1e6 <= theta)
    @constraint(main, sum(a) <= 500)
    @objective(main, Min, 150*a[1] + 230*a[2] + 260*a[3] + theta)
    return main
end

function solve_farmer_main(main::Model)
    
    optimize!(main)
    obj = objective_value(main)
    a = value.(main[:a])
    return obj, a
end

function create_farmer_subproblem()
    
    subproblem = Model(() -> ParametricOptInterface.Optimizer(HiGHS.Optimizer()))
    MOI.set(subproblem, JuMP.MOI.Silent(), true)
    
    @variable(subproblem, a[1:3] in ParametricOptInterface.Parameter(0))
    @variable(subproblem, rate[1:3] in ParametricOptInterface.Parameter(0))
    @variable(subproblem, a_aux[1:3])
    @constraint(subproblem, dual_fisher[i = 1:3] ,a[i] == a_aux[i])
    @variable(subproblem, 0<=c[1:2])
    @variable(subproblem, 0<=v[1:2])
    @variable(subproblem, 0<=l)
    @expression(subproblem, p[i=1:3], a_aux[i]*rate[i])
    @constraint(subproblem, p[1] + c[1] - v[1] >= 200 )
    @constraint(subproblem, p[2] + c[2] - v[2] >= 240 )
    @constraint(subproblem, l <= 36*p[3] )
    @constraint(subproblem, l <= 10*(p[3]-6000) + 36*6000)
    @objective(subproblem, Min, 238*c[1] + 210*c[2] - 170*v[1] - 150*v[2] - l)

    return subproblem
end

function solve_farmer_subproblem(subproblem::Model, a::Vector{Float64})
    
    rate = [2 2.5 3
            2.4 3 3.6
            16 20 24]
    pis = zeros(3,3)
    objs = zeros(3)
    
    MOI.set.(subproblem, POI.ParameterValue(), subproblem[:a], a)
    for i in 1:3
        MOI.set.(subproblem, POI.ParameterValue(), subproblem[:rate], rate[:,i])
        optimize!(subproblem)

        objs[i] = objective_value(subproblem)
        pis[i,:] = dual.(subproblem[:dual_fisher])
    end
    obj = sum(objs*(1/3))    
    pi = reshape(sum(pis*(1/3), dims=1), 3)
    return obj, pi
end

function update_main!(main::Model, a::Vector{Float64}, pi::Vector{Float64}, obj_sub::Float64)
    @constraint(main, main[:theta] >= obj_sub - sum(pi[i]*(main[:a][i] - a[i]) for i in 1:3))
    nothing
end

function farmer_bender()
    c = [150, 230,260]
    main = create_farmer_main()
    subproblem = create_farmer_subproblem()
    obj_main, a = solve_farmer_main(main)
    obj_sub, pi = solve_farmer_subproblem(subproblem, a)
    LB = obj_main
    UB = obj_sub + c'a
    gap = UB - LB
    while gap >= 1e-6
        update_main!(main, a, pi, obj_sub)
        obj_main, a = solve_farmer_main(main)
        obj_sub, pi = solve_farmer_subproblem(subproblem, a)
        LB = obj_main
        UB = min(obj_sub + c'a , UB)
        @show gap = UB - LB
    end
    return main
end

farmer_bender (generic function with 1 method)

In [24]:
main = farmer_bender();
value.(main[:a])

gap = UB - LB = 103999.99999999994
gap = UB - LB = 30800.0
gap = UB - LB = 22647.058823529427
gap = UB - LB = 22077.535545023668
gap = UB - LB = 5539.961315280467
gap = UB - LB = 960.0000000000291
gap = UB - LB = 853.8461538461561
gap = UB - LB = 471.11111111112405
gap = UB - LB = 412.1367521367356
gap = UB - LB = -1.4551915228366852e-11


3-element Vector{Float64}:
 170.0
  80.00000000000017
 249.99999999999983